In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, mannwhitneyu, levene

ile path

In [ ]:
input_file = "1_features.csv"
output_folder = "graphs"
output_summary_file = "summary_statistics.csv"
output_test_file = "statistical_tests.csv"

eading files

In [ ]:
os.makedirs(output_folder, exist_ok=True)

In [ ]:
df = pd.read_csv(input_file, index_col=1)

In [ ]:
letters = list("abcdefghijklmnopqrstuvwxyz")
non_letters = ['sad', 'smiley', 'xclaim']

eperating letetrer from non-letetr

In [ ]:
df['type'] = df['label'].apply(lambda x: 'Letter' if x in letters else 'Non-Letter')

ames for columns

In [ ]:
columns = ["nr_pix", "rows_with_1", "cols_with_1", 
           "rows_with_3p", "cols_with_3p", "aspect_ratio", "density"]

In [ ]:
titles = ["nr_pix", "rows_with_1", "cols_with_1", "rows_with_3p", "cols_with_3p", "aspect_ratio", "density"]

In [ ]:
print(columns)

reating histograms

In [ ]:
for col, title in zip(columns, titles):
    plt.figure(figsize=(8, 6))
    sns.histplot(df[col], kde=True, color='skyblue')
    plt.title(f"{title} Histogram")
    plt.xlabel(title)
    plt.ylabel('Frequency')

    # Save the plot
    output_path = os.path.join(output_folder, f"{title}_histogram.png")
    plt.savefig(output_path)
    plt.close()

xcludes index

In [ ]:
features = df.drop(columns=['label', 'type']).select_dtypes(include=['float64', 'int64']).columns

ean, median, and standard deviation for letters

In [ ]:
summary_letters = df[df['type'] == 'Letter'][features].agg(['mean', 'median', 'std'])

ean, median, and standard deviation for non-letters

In [ ]:
summary_non_letters = df[df['type'] == 'Non-Letter'][features].agg(['mean', 'median', 'std'])

reating table

In [ ]:
summary_combined = pd.concat([summary_letters, summary_non_letters], keys=['Letters', 'Non-Letters'])

In [ ]:
summary_combined.to_csv(output_summary_file)

lotting histograms

In [ ]:
interesting_features = ['nr_pix', 'no_neigh_below', 'eyes']

In [ ]:
for feature in interesting_features:
    plt.figure(figsize=(8, 6))
    sns.histplot(df[df['type'] == 'Letter'][feature], color='blue', label='Letters', kde=True, stat="density", bins=30)
    sns.histplot(df[df['type'] == 'Non-Letter'][feature], color='red', label='Non-Letters', kde=True, stat="density", bins=30)
    
    plt.title(f"{feature} Distribution for Letters vs Non-Letters")
    plt.xlabel(feature)
    plt.ylabel("Density")
    plt.legend()
    
    # Save the plot
    plt.savefig(f"graphs/{feature}_comparison.png")
    plt.close()

tatistical analysis

In [ ]:
results = []

In [ ]:
for feature in features:
    #splitting data in non-letters and letters
    data_letters = df[df['type'] == 'Letter'][feature]
    data_non_letters = df[df['type'] == 'Non-Letter'][feature]
    
    #levens tests
    _, p_var = levene(data_letters, data_non_letters)
    
    if p_var < 0.05:
        # If variances are unequal, use Mann-Whitney U test
        stat, p_value = mannwhitneyu(data_letters, data_non_letters, alternative='two-sided')
        test_used = 'Mann-Whitney U'
    else:
        #t test
        stat, p_value = ttest_ind(data_letters, data_non_letters, equal_var=True)
        test_used = 'T-test'
    
    #storing the result.
    results.append({
        'Feature': feature,
        'Test Used': test_used,
        'p-value': p_value,
        'Significant': p_value < 0.05  # If p-value < 0.05, it's significant
    })

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df.to_csv(output_test_file, index=False)

electing most useful features based on statistical significance

In [ ]:
significant_features = results_df[results_df['Significant']]['Feature'].tolist()

In [ ]:
for feature in significant_features:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x='type', y=feature, data=df)
    plt.title(f"{feature} Distribution for Letters vs Non-Letters")
    plt.xlabel("Type")
    plt.ylabel(feature)
    plt.savefig(os.path.join(output_folder, f"{feature}_boxplot.png"))
    plt.close()

In [ ]:
features_to_plot = ['nr_pix', 'eyes']

In [ ]:
sns.pairplot(df, vars=features_to_plot, hue='type', palette={'Letter': 'blue', 'Non-Letter': 'red'})
plt.suptitle('Feature Scatter Plots for Letters vs Non-Letters', y=1.02)

In [ ]:
plt.savefig('graphs/scatter_plot.png')
plt.close()

In [ ]:
print(f"\nBoxplots saved to '{output_folder}'!")

In [ ]:
for feature in significant_features:
    print(f"- {feature}")